In [1]:
"""
Manhattan Summer PM Pedestrian Activity Heatmap
Creates a focused heatmap showing pedestrian activity in Manhattan during summer PM hours
"""

'\nManhattan Summer PM Pedestrian Activity Heatmap\nCreates a focused heatmap showing pedestrian activity in Manhattan during summer PM hours\n'

In [2]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from shapely.geometry import Point
import warnings

In [3]:
warnings.filterwarnings("ignore")

In [7]:
def create_manhattan_summer_pm_heatmap():
    """Create Manhattan summer PM pedestrian activity heatmap."""

    print("🔥 Creating Manhattan Summer PM Pedestrian Activity Heatmap...")

    # Load pedestrian data
    ped_df = pd.read_csv("Data/Bi-Annual_Pedestrian_Counts_20250709.csv")

    # Extract coordinates from WKT format
    ped_df["Longitude"] = (
        ped_df["the_geom"].str.extract(r"POINT \(([-\d.]+)")[0].astype(float)
    )
    ped_df["Latitude"] = ped_df["the_geom"].str.extract(r"([-\d.]+)\)")[0].astype(float)

    # Create GeoDataFrame for pedestrian data
    ped_geometry = [Point(xy) for xy in zip(ped_df["Longitude"], ped_df["Latitude"])]
    ped_gdf = gpd.GeoDataFrame(ped_df, geometry=ped_geometry, crs="EPSG:4326")

    # Load MapPLUTO to get Manhattan bounds
    pluto_gdf = gpd.read_file("Data/nyc_mappluto_25v2_shp/MapPLUTO.shp")
    manhattan_pluto = pluto_gdf[pluto_gdf["Borough"] == "MN"].copy()
    manhattan_pluto_wgs84 = manhattan_pluto.to_crs("EPSG:4326")
    manhattan_bounds = manhattan_pluto_wgs84.total_bounds

    # Filter pedestrian data to Manhattan bounds
    manhattan_ped = ped_gdf[
        (ped_gdf["Longitude"] >= manhattan_bounds[0])
        & (ped_gdf["Longitude"] <= manhattan_bounds[2])
        & (ped_gdf["Latitude"] >= manhattan_bounds[1])
        & (ped_gdf["Latitude"] <= manhattan_bounds[3])
    ].copy()

    print(f"   Manhattan pedestrian locations: {len(manhattan_ped)}")